In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

from sklearn.model_selection import KFold
from tqdm import tqdm
import gc
import datetime
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, roc_curve, auc, roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score,precision_recall_curve,roc_curve, recall_score,precision_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500

In [2]:
# Path = '../input/amexml/'
Path = '../input/amex-basic-processing/'
train_df=pd.read_hdf(Path+'train_basic.hdf')
print('train reading complete')
test_df=pd.read_hdf(Path+'test_basic.hdf')
print('test reading complete')
# log_df=pd.read_hdf('../input/amex-classification/'+'historical_user_logs.hdf')
# print('log reading complete')

train reading complete
test reading complete


In [3]:
print(train_df.shape)
train_df.head()

(463291, 74)


,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,day,dayofweek,month,hour,action_mean,action_sum,action_count,action_month_mean,action_month_sum,action_month_count,action_dayofweek_mean,action_dayofweek_sum,action_dayofweek_count,action_hour_mean,action_hour_sum,action_hour_count,action_usermonth_mean,action_usermonth_sum,action_usermonth_count,action_userdayofweek_mean,action_userdayofweek_sum,action_userdayofweek_count,action_userhour_mean,action_userhour_sum,action_userhour_count,action_productmonth_mean,action_productmonth_sum,action_productmonth_count,action_productdayofweek_mean,action_productdayofweek_sum,action_productdayofweek_count,action_producthour_mean,action_producthour_sum,action_producthour_count,action_userproductmonth_mean,action_userproductmonth_sum,action_userproductmonth_count,action_userproductdayofweek_mean,action_userproductdayofweek_sum,action_userproductdayofweek_count,action_userproducthour_mean,action_userproducthour_sum,action_userproducthour_count,action_viewsum,action_month_viewsum,action_dayofweek_viewsum,action_hour_viewsum,action_usermonth_viewsum,action_userdayofweek_viewsum,action_userhour_viewsum,action_productmonth_viewsum,action_productdayofweek_viewsum,action_producthour_viewsum,action_userproductmonth_viewsum,action_userproductdayofweek_viewsum,action_userproducthour_viewsum,action_userproduct_date_mean_total,action_product_date_mean_total,action_user_date_mean_total
0,140690,2017-07-02 00:00:00,858557,0,359520,13787,4,NaN,10.0,0,4.0,3.0,3.0,0,0,2,6,7,0,0.000000,0.0,5.0,0.017626,32672,1853670,0.018257,62319,3413502,0.017553,2729,155471,0.000000,0.0,198.0,0.0,0.0,129.0,0.0,0.0,35.0,0.017464,2333,133591,0.017825,4245,238151,0.018553,242,13044,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,1820998,3351183,152742,198.0,129.0,35.0,131258,233906,12802,0.0,0.0,0.0,-3.0,0.053841,0.000000
1,333291,2017-07-02 00:00:00,243253,0,105960,11085,5,NaN,8.0,0,2.0,2.0,NaN,0,0,2,6,7,0,0.000000,0.0,11.0,0.017626,32672,1853670,0.018257,62319,3413502,0.017553,2729,155471,-1.000000,-1.0,-1.0,0.0,0.0,42.0,-1.0,-1.0,-1.0,0.017464,2333,133591,0.017825,4245,238151,0.018553,242,13044,-1.0,-1.0,-1.0,0.0,0.0,10.0,-1.0,-1.0,-1.0,11.0,1820998,3351183,152742,0.0,42.0,0.0,131258,233906,12802,0.0,10.0,0.0,-2.0,0.053841,-2.000000
2,129781,2017-07-02 00:00:00,243253,0,359520,13787,4,NaN,8.0,0,2.0,2.0,NaN,0,0,2,6,7,0,0.000000,0.0,11.0,0.017626,32672,1853670,0.018257,62319,3413502,0.017553,2729,155471,-1.000000,-1.0,-1.0,0.0,0.0,42.0,-1.0,-1.0,-1.0,0.017464,2333,133591,0.017825,4245,238151,0.018553,242,13044,-1.0,-1.0,-1.0,0.0,0.0,10.0,-1.0,-1.0,-1.0,11.0,1820998,3351183,152742,0.0,42.0,0.0,131258,233906,12802,0.0,10.0,0.0,-2.0,0.053841,-2.000000
3,464848,2017-07-02 00:00:00,1097446,1,359520,13787,3,NaN,3.0,1,3.0,3.0,2.0,1,0,2,6,7,0,0.111111,1.0,9.0,0.017626,32672,1853670,0.018257,62319,3413502,0.017553,2729,155471,0.000000,0.0,2.0,0.0,0.0,8.0,-1.0,-1.0,-1.0,0.017303,2297,132748,0.017131,4037,235660,0.017887,182,10175,-1.0,-1.0,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,8.0,1820998,3351183,152742,2.0,8.0,0.0,130451,231623,9993,0.0,1.0,0.0,-2.0,0.052321,-1.000000
4,90569,2017-07-02 00:01:00,663656,0,405490,60305,3,NaN,2.0,1,2.0,3.0,2.0,1,0,2,6,7,0,0.000000,0.0,3.0,0.017626,32672,1853670,0.018257,62319,3413502,0.017553,2729,155471,0.043478,2.0,46.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.017464,2333,133591,0.017825,4245,238151,0.018553,242,13044,0.0,0.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,1820998,3351183,152742,44.0,0.0,0.0,131258,233906,12802,3.0,0.0,0.0,-2.0,0.053841,-1.956522


In [4]:
print(test_df.shape)
print(100 *test_df.shape[0] / train_df.shape[0])
test_df.head()
#train to test ratio


(128858, 73)
27.813620381142737


,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,day,dayofweek,month,hour,action_mean,action_sum,action_count,action_month_mean,action_month_sum,action_month_count,action_dayofweek_mean,action_dayofweek_sum,action_dayofweek_count,action_hour_mean,action_hour_sum,action_hour_count,action_usermonth_mean,action_usermonth_sum,action_usermonth_count,action_userdayofweek_mean,action_userdayofweek_sum,action_userdayofweek_count,action_userhour_mean,action_userhour_sum,action_userhour_count,action_productmonth_mean,action_productmonth_sum,action_productmonth_count,action_productdayofweek_mean,action_productdayofweek_sum,action_productdayofweek_count,action_producthour_mean,action_producthour_sum,action_producthour_count,action_userproductmonth_mean,action_userproductmonth_sum,action_userproductmonth_count,action_userproductdayofweek_mean,action_userproductdayofweek_sum,action_userproductdayofweek_count,action_userproducthour_mean,action_userproducthour_sum,action_userproducthour_count,action_viewsum,action_month_viewsum,action_dayofweek_viewsum,action_hour_viewsum,action_usermonth_viewsum,action_userdayofweek_viewsum,action_userhour_viewsum,action_productmonth_viewsum,action_productdayofweek_viewsum,action_producthour_viewsum,action_userproductmonth_viewsum,action_userproductdayofweek_viewsum,action_userproducthour_viewsum,action_userproduct_date_mean_total,action_product_date_mean_total,action_user_date_mean_total
0,411705,2017-07-08,732573,8,404347,53587,1,NaN,5.0,1,5.0,3.0,NaN,0,8,5,7,0,-1.00,-1.0,-1.0,0.017626,32672,1853670,0.0178,59849,3362265,0.017553,2729,155471,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.014201,24,1690,0.013514,45,3330,0.017182,5,291,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1820998,3302416,152742,0.0,0.0,0.0,1666,3285,286,0.0,0.0,0.0,-6.0,0.089794,-6.0
1,208263,2017-07-08,172910,1,118601,28529,3,82527.0,NaN,-1,NaN,NaN,NaN,1,8,5,7,0,0.25,1.0,4.0,0.017626,32672,1853670,0.0178,59849,3362265,0.017553,2729,155471,0.0,0.0,14.0,0.0,0.0,14.0,-1.0,-1.0,-1.0,0.017303,2297,132748,0.017308,4144,239433,0.017887,182,10175,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,1820998,3302416,152742,14.0,14.0,0.0,130451,235289,9993,0.0,0.0,0.0,-6.0,0.104996,-2.0
2,239450,2017-07-08,172910,1,118601,28529,4,82527.0,NaN,-1,NaN,NaN,NaN,1,8,5,7,0,0.25,1.0,4.0,0.017626,32672,1853670,0.0178,59849,3362265,0.017553,2729,155471,0.0,0.0,14.0,0.0,0.0,14.0,-1.0,-1.0,-1.0,0.017303,2297,132748,0.017308,4144,239433,0.017887,182,10175,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,1820998,3302416,152742,14.0,14.0,0.0,130451,235289,9993,0.0,0.0,0.0,-6.0,0.104996,-2.0
3,547761,2017-07-08,557318,6,118601,28529,5,82527.0,1.0,1,1.0,3.0,1.0,0,8,5,7,0,-1.00,-1.0,-1.0,0.017626,32672,1853670,0.0178,59849,3362265,0.017553,2729,155471,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,10.0,0.016383,2123,129585,0.016652,3954,237449,0.015185,159,10471,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1820998,3302416,152742,0.0,0.0,10.0,127462,233495,10312,0.0,0.0,0.0,-6.0,0.096440,-4.0
4,574275,2017-07-08,923896,3,118601,28529,5,82527.0,9.0,0,3.0,1.0,NaN,1,8,5,7,0,-1.00,-1.0,-1.0,0.017626,32672,1853670,0.0178,59849,3362265,0.017553,2729,155471,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.014321,9026,630264,0.014355,16090,1120868,0.014605,821,56212,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1820998,3302416,152742,0.0,0.0,0.0,621238,1104778,55391,0.0,0.0,0.0,-6.0,0.086563,-6.0


In [7]:
targetcol='is_click'

In [8]:
train_df[targetcol] = train_df[targetcol].astype('int32')

In [9]:
train_df.dtypes

session_id                                      int64
DateTime                               datetime64[ns]
user_id                                         int64
product                                         int64
campaign_id                                     int64
webpage_id                                      int64
product_category_1                              int64
product_category_2                            float64
user_group_id                                 float64
gender                                          int64
age_level                                     float64
user_depth                                    float64
city_development_index                        float64
var_1                                           int64
is_click                                        int32
day                                             int64
dayofweek                                       int64
month                                           int64
hour                        

In [10]:
def getenccolname(colname):
    return "targetenc_"+colname
#     return "targetvarenc_"+colname
#Target Encoding on categorical features
def targetencode(train,test,catcolnames,targetcolname):
    #Target Encoding
#     if train.index[0]!=0:
#         print('Caution: train index will be reset')
#         train.reset_index(inplace=True)
#         del train['index']
#     if test.index[0]!=0:
#         print('Caution: test index will be reset')
#         test.reset_index(inplace=True)
#         del test['index']
        
    cols_agg = ['mean']
#     cols_agg = ['var']
    mean_col_index = 0

    kfdata = KFold(n_splits = 5, shuffle = False, random_state = 123)
    labels = train[targetcolname]
    indices= kfdata.split(train,labels)
#     kf = KFold(n_splits=5, shuffle=False,random_state=123)
    # kfdata= kf.split(all_data,all_data[cols_target].values)

    fold=1
    for train_rowno, val_rowno in tqdm(indices):
        print("******************")
        print("FOLD:",fold)
        print("******************")
        X_tr, X_val = train.iloc[train_rowno],train.iloc[val_rowno] 
        train_index = X_tr.index
        val_index = X_val.index
        print('valid index:', val_index)
        print('train index:', train_index)
       # In our non-regularized case we just *map* the computed means to the `item_id`'s
        for curcol in tqdm(catcolnames):
                islisttype = (type(curcol)==type([]))
                print('curcol=',curcol)
                otherfolds_enc = X_tr.groupby(curcol)[targetcolname].agg(cols_agg)
                if (islisttype):
                    cols = '_'.join(curcol)
                else:
                    cols = curcol
                enccol = getenccolname(cols)
                otherfolds_enc.columns = [enccol]
                # Assign the new encoded columns (mean, min, max) to training set by 
                # performing mapping on the group by columns on the target encoded data
                if (islisttype):
                      # caution: the below code resets the index of train data
                    abcdefgh = 100
#                     train[val_index]= pd.merge(train[val_index].reset_index(drop=True),otherfolds_enc.reset_index(drop=True),on=curcol,how='left')
                else:
#                     train.iloc[val_index][enccol] = X_val[curcol].map(otherfolds_enc[enccol])
                    train.loc[val_index, enccol] = X_val[curcol].map(otherfolds_enc[enccol])
                del otherfolds_enc
                gc.collect()
        del X_tr,X_val
        gc.collect() 
        fold+=1
    

    enc_colnames=[getenccolname(col) for col in catcolnames]

    print()
    print('Fill NA Start')
    # Fill NaNs of rare categories of train with global mean
    for i,enccol in enumerate(enc_colnames):
        islisttype = (type(curcol)==type([]))
        global_mean= train[enccol].mean()
        train[enccol].fillna(global_mean, inplace=True) 
#         print('i:{} test cur col:{}'.format(i,enccol))
    #     # Populate mean encoding for test data set by mapping the train data set target mean encoding of 
    #     # corresponding group by col 
        curcol = catcolnames[i]
        train_colenc_means = train.groupby(catcolnames[i])[enccol].mean()
        test[enccol]= test[catcolnames[i]].map(train_colenc_means)
        test[enccol].fillna(global_mean,inplace=True)

    print(train.shape)
    print()
    print('Target Encoding completed')

In [11]:
def genfeats_fortargetenc(df):
    newfeat = 'user_id#product' 
    df[newfeat] = df['user_id'].astype(str).str.cat(df['product'].astype(str),"_")
    return df

In [12]:
train_df=genfeats_fortargetenc(train_df)
test_df=genfeats_fortargetenc(test_df)

In [13]:
#Label Encoding of new interaction feats
new_feats= ['user_id#product']
for col in new_feats:
    print('col=',col)
    train_df[col], indexer = pd.factorize(train_df[col])
    test_df[col] = indexer.get_indexer(test_df[col])

col= user_id#product


In [27]:
#Target Encoding
cat_cols=['product','gender','user_id',  'campaign_id', 'webpage_id',
'product_category_1','product_category_2','user_group_id','var_1']
action_interestcount_cols = [col for col in train_df.columns if ('_sum' in col) and ('action_' in col)]
action_viewcount_cols = [col for col in train_df.columns if ('_viewsum' in col) and ('action_' in col)]

non_cat_cols =['dayofweek','month','hour']

action_viewandinteresttotal_cols = [col for col in train_df.columns if ('_count' in col) and ('action_' in col)]

cat_cols+= action_viewcount_cols + action_interestcount_cols  + new_feats + non_cat_cols

targetencode(train_df,test_df,cat_cols,targetcol)

0it [00:00, ?it/s]

******************
FOLD: 1
******************



  0%|          | 0/39 [00:00<?, ?it/s]

valid index: Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            92649, 92650, 92651, 92652, 92653, 92654, 92655, 92656, 92657,
            92658],
           dtype='int64', length=92659)
train index: Int64Index([ 92659,  92660,  92661,  92662,  92663,  92664,  92665,  92666,
             92667,  92668,
            ...
            463281, 463282, 463283, 463284, 463285, 463286, 463287, 463288,
            463289, 463290],
           dtype='int64', length=370632)
curcol= product



  3%|▎         | 1/39 [00:00<00:13,  2.91it/s]

...


 95%|█████████▍| 37/39 [00:04<00:00,  8.61it/s]

curcol= dayofweek
curcol= month



100%|██████████| 39/39 [00:04<00:00,  9.12it/s]
5it [00:35,  7.48s/it]

curcol= hour



Fill NA Start
(463291, 114)

Target Encoding completed


In [28]:
noof1s=train_df[train_df[targetcol]!=0].shape[0]
noof0s=train_df[train_df[targetcol]==0].shape[0]
onesratio = noof1s / (noof0s+noof1s)
zerosratio = noof0s / (noof0s+noof1s)
print('noof1s=',noof1s)
print('noof0s=',noof0s)
print('1sratio=',onesratio)
print('0sratio=',zerosratio)

noof1s= 31331
noof0s= 431960
1sratio= 0.06762704218299082
0sratio= 0.9323729578170091


In [26]:
# #drop target enc columns
# enc_cols = [col for col in train_df.columns if ('targetenc' in col) ]
# train_df.drop(enc_cols,inplace=True,axis=1)
# test_df.drop(enc_cols,inplace=True,axis=1)

In [29]:
#Add Total Target Encoding Field
    
enc_cols = [col for col in train_df.columns if ('targetenc' in col) ]
for col in enc_cols:
    train_df[col] = train_df[col] / onesratio
    test_df[col] = test_df[col] / onesratio

In [77]:
enc_cols = [col for col in train_df.columns if ('targetenc' in col) ]
train_df[enc_cols].head()

,targetenc_product,targetenc_gender,targetenc_user_id,targetenc_campaign_id,targetenc_webpage_id,targetenc_product_category_1,targetenc_product_category_2,targetenc_user_group_id,targetenc_var_1,targetenc_action_viewsum,targetenc_action_month_viewsum,targetenc_action_dayofweek_viewsum,targetenc_action_hour_viewsum,targetenc_action_usermonth_viewsum,targetenc_action_userdayofweek_viewsum,targetenc_action_userhour_viewsum,targetenc_action_productmonth_viewsum,targetenc_action_productdayofweek_viewsum,targetenc_action_producthour_viewsum,targetenc_action_userproductmonth_viewsum,targetenc_action_userproductdayofweek_viewsum,targetenc_action_userproducthour_viewsum,targetenc_action_sum,targetenc_action_month_sum,targetenc_action_dayofweek_sum,targetenc_action_hour_sum,targetenc_action_usermonth_sum,targetenc_action_userdayofweek_sum,targetenc_action_userhour_sum,targetenc_action_productmonth_sum,targetenc_action_productdayofweek_sum,targetenc_action_producthour_sum,targetenc_action_userproductmonth_sum,targetenc_action_userproductdayofweek_sum,targetenc_action_userproducthour_sum,targetenc_user_id#product,targetenc_dayofweek,targetenc_month,targetenc_hour
0,0.993663,0.946433,0.000000,0.860322,0.791184,0.860867,0.934755,0.826426,0.945774,0.979018,0.980341,0.989111,0.963179,2.274921,0.445391,0.891677,0.993663,0.986465,0.946087,0.970413,0.972537,0.970904,0.989233,0.980341,0.989111,0.963179,0.970569,0.984143,0.967481,0.993663,0.974327,0.946087,0.970397,0.972603,0.970867,0.895520,0.989111,0.980341,0.963179
1,0.993663,0.946433,0.938229,1.036963,1.036963,0.882178,0.934755,0.962647,0.945774,0.955232,0.980341,0.989111,0.963179,0.985024,0.926034,0.996699,0.993663,0.986465,0.946087,0.970413,1.079050,0.970904,0.989233,0.980341,0.989111,0.963179,0.984823,0.984143,0.996947,0.993663,0.974327,0.946087,0.970397,1.002252,0.970867,0.895520,0.989111,0.980341,0.963179
2,0.993663,0.946433,0.938229,0.860322,0.791184,0.860867,0.934755,0.962647,0.945774,0.955232,0.980341,0.989111,0.963179,0.985024,0.926034,0.996699,0.993663,0.986465,0.946087,0.970413,1.079050,0.970904,0.989233,0.980341,0.989111,0.963179,0.984823,0.984143,0.996947,0.993663,0.974327,0.946087,0.970397,1.002252,0.970867,0.895520,0.989111,0.980341,0.963179
3,0.907613,0.985118,0.000000,0.860322,0.791184,1.180556,0.934755,0.951867,1.026934,0.984949,0.980341,0.989111,0.963179,0.952336,0.974005,0.996699,0.907613,0.986465,0.802394,0.970413,0.926720,0.970904,0.942415,0.980341,0.989111,0.963179,0.970569,0.984143,0.996947,0.907613,0.585347,0.858599,0.970397,1.002252,0.970867,0.000000,0.989111,0.980341,0.963179
4,0.993663,0.985118,2.957397,1.280554,1.280554,1.180556,0.934755,1.026805,1.026934,0.959263,0.980341,0.989111,0.963179,1.078513,0.982004,0.996699,0.993663,0.986465,0.946087,1.015467,0.972537,0.970904,0.989233,0.980341,0.989111,0.963179,1.052332,0.982999,0.996947,0.993663,0.974327,0.946087,1.004926,0.972603,0.970867,14.786984,0.989111,0.980341,0.963179


In [71]:
# summarize_cols = ['sum_targetenc','product_targetenc','mean_targetenc']
# train_df.drop(summarize_cols,inplace=True,axis=1)
# test_df.drop(summarize_cols,inplace=True,axis=1)

In [72]:
enc_cols_for_sum = [col for col in train_df.columns if ('targetenc' in col) & ('#' not in col)
                   & ('action_' not in col)]
print(enc_cols_for_sum)
# enc_cols_for_sum= cat_cols + non_cat_cols

#View and Interest category target enc
enc_cols_for_sum += ['targetenc_action_viewsum','targetenc_action_sum']

train_df['sum_targetenc'] = 0
test_df['sum_targetenc'] = 0
train_df['product_targetenc'] = 1
test_df['product_targetenc'] = 1
for col in enc_cols_for_sum:
    train_df['product_targetenc'] *=(train_df[col]+1)
    test_df['product_targetenc'] *=(test_df[col]+1)
    
    train_df['sum_targetenc'] +=train_df[col]
    test_df['sum_targetenc'] +=test_df[col]
    
train_df['mean_targetenc'] = train_df['sum_targetenc'] / len(enc_cols_for_sum)
test_df['mean_targetenc'] = test_df['sum_targetenc'] / len(enc_cols_for_sum)

['targetenc_product', 'targetenc_gender', 'targetenc_user_id', 'targetenc_campaign_id', 'targetenc_webpage_id', 'targetenc_product_category_1', 'targetenc_product_category_2', 'targetenc_user_group_id', 'targetenc_var_1', 'targetenc_dayofweek', 'targetenc_month', 'targetenc_hour']


In [73]:
len(enc_cols)
train_df[['sum_targetenc','product_targetenc','mean_targetenc']+ enc_cols_for_sum ].describe()

,sum_targetenc,product_targetenc,mean_targetenc,targetenc_product,targetenc_gender,targetenc_user_id,targetenc_campaign_id,targetenc_webpage_id,targetenc_product_category_1,targetenc_product_category_2,targetenc_user_group_id,targetenc_var_1,targetenc_dayofweek,targetenc_month,targetenc_hour,targetenc_action_viewsum,targetenc_action_sum
count,463291.000000,4.632910e+05,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000,463291.000000
mean,13.853224,1.592646e+04,0.989516,1.000039,0.999912,0.938229,0.995427,0.995653,0.998658,0.934755,0.998465,0.999985,0.989111,1.000000,1.002597,1.000153,1.000239
std,2.188899,2.012799e+04,0.156350,0.100505,0.029081,2.061968,0.223085,0.215790,0.147309,0.066988,0.081753,0.047862,0.081077,0.022801,0.057102,0.174233,0.052469
min,10.096170,1.266065e+03,0.721155,0.655652,0.922822,0.000000,0.636736,0.760406,0.845103,0.000000,0.821426,0.933699,0.835747,0.969726,0.642248,0.000000,0.000000
25%,12.745889,7.193353e+03,0.910421,0.964681,0.974257,0.000000,0.840383,0.805119,0.882178,0.934755,0.947431,0.966371,0.920020,0.980341,0.965105,0.956714,0.977452
50%,13.389879,1.073740e+04,0.956420,1.022948,1.003827,0.000000,0.935327,0.935327,0.908973,0.934755,0.993441,0.992863,0.989111,0.999369,1.000928,0.994983,0.999790
75%,14.156349,1.619961e+04,1.011168,1.049443,1.027443,0.938229,1.146544,1.146544,1.074001,0.934755,1.066605,1.045312,1.051403,1.022310,1.038769,1.017108,1.027026
max,42.467079,1.401939e+06,3.033363,1.402668,1.080242,14.786984,1.415754,1.415754,1.307378,1.945656,1.473373,1.077384,1.125716,1.028254,1.168594,14.786984,14.786984


In [76]:
sum_targetenc_thres = 5000
mask1=train_df['product_targetenc']<=sum_targetenc_thres

print(train_df.shape)
print(train_df[(mask1)].shape)
print(train_df[(mask1) & (train_df[targetcol]!=0)].shape)
print(train_df[(mask1) & (train_df[targetcol]==0)].shape)

(463291, 117)
(12370, 117)
(563, 117)
(11807, 117)


In [79]:
enc_cols = [col for col in train_df.columns if ('targetenc' in col) ]
train_df[enc_cols].head()

,targetenc_product,targetenc_gender,targetenc_user_id,targetenc_campaign_id,targetenc_webpage_id,targetenc_product_category_1,targetenc_product_category_2,targetenc_user_group_id,targetenc_var_1,targetenc_action_viewsum,targetenc_action_month_viewsum,targetenc_action_dayofweek_viewsum,targetenc_action_hour_viewsum,targetenc_action_usermonth_viewsum,targetenc_action_userdayofweek_viewsum,targetenc_action_userhour_viewsum,targetenc_action_productmonth_viewsum,targetenc_action_productdayofweek_viewsum,targetenc_action_producthour_viewsum,targetenc_action_userproductmonth_viewsum,targetenc_action_userproductdayofweek_viewsum,targetenc_action_userproducthour_viewsum,targetenc_action_sum,targetenc_action_month_sum,targetenc_action_dayofweek_sum,targetenc_action_hour_sum,targetenc_action_usermonth_sum,targetenc_action_userdayofweek_sum,targetenc_action_userhour_sum,targetenc_action_productmonth_sum,targetenc_action_productdayofweek_sum,targetenc_action_producthour_sum,targetenc_action_userproductmonth_sum,targetenc_action_userproductdayofweek_sum,targetenc_action_userproducthour_sum,targetenc_user_id#product,targetenc_dayofweek,targetenc_month,targetenc_hour,sum_targetenc,product_targetenc,mean_targetenc
0,0.993663,0.946433,0.000000,0.860322,0.791184,0.860867,0.934755,0.826426,0.945774,0.979018,0.980341,0.989111,0.963179,2.274921,0.445391,0.891677,0.993663,0.986465,0.946087,0.970413,0.972537,0.970904,0.989233,0.980341,0.989111,0.963179,0.970569,0.984143,0.967481,0.993663,0.974327,0.946087,0.970397,0.972603,0.970867,0.895520,0.989111,0.980341,0.963179,12.060306,5036.742216,0.861450
1,0.993663,0.946433,0.938229,1.036963,1.036963,0.882178,0.934755,0.962647,0.945774,0.955232,0.980341,0.989111,0.963179,0.985024,0.926034,0.996699,0.993663,0.986465,0.946087,0.970413,1.079050,0.970904,0.989233,0.980341,0.989111,0.963179,0.984823,0.984143,0.996947,0.993663,0.974327,0.946087,0.970397,1.002252,0.970867,0.895520,0.989111,0.980341,0.963179,13.554700,13053.484380,0.968193
2,0.993663,0.946433,0.938229,0.860322,0.791184,0.860867,0.934755,0.962647,0.945774,0.955232,0.980341,0.989111,0.963179,0.985024,0.926034,0.996699,0.993663,0.986465,0.946087,0.970413,1.079050,0.970904,0.989233,0.980341,0.989111,0.963179,0.984823,0.984143,0.996947,0.993663,0.974327,0.946087,0.970397,1.002252,0.970867,0.895520,0.989111,0.980341,0.963179,13.110970,10364.378751,0.936498
3,0.907613,0.985118,0.000000,0.860322,0.791184,1.180556,0.934755,0.951867,1.026934,0.984949,0.980341,0.989111,0.963179,0.952336,0.974005,0.996699,0.907613,0.986465,0.802394,0.970413,0.926720,0.970904,0.942415,0.980341,0.989111,0.963179,0.970569,0.984143,0.996947,0.907613,0.585347,0.858599,0.970397,1.002252,0.970867,0.000000,0.989111,0.980341,0.963179,12.498343,6279.688559,0.892739
4,0.993663,0.985118,2.957397,1.280554,1.280554,1.180556,0.934755,1.026805,1.026934,0.959263,0.980341,0.989111,0.963179,1.078513,0.982004,0.996699,0.993663,0.986465,0.946087,1.015467,0.972537,0.970904,0.989233,0.980341,0.989111,0.963179,1.052332,0.982999,0.996947,0.993663,0.974327,0.946087,1.004926,0.972603,0.970867,14.786984,0.989111,0.980341,0.963179,16.547462,42551.055745,1.181962


In [ ]:
train_df.to_hdf('train_targetenc.hdf',key='data')
print('train_df save complete')
test_df.to_hdf('test_targetenc.hdf',key='data')
print('test_df save complete')